# Volume 2, Chapter 13: Network Documentation Basics

**Auto-generate always-accurate documentation from network configs using Claude AI**

From: AI for Networking Engineers - Volume 2, Chapter 13

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/main/CODE/Colab-Notebooks/Vol2_Ch13_Network_Documentation.ipynb)

## What You'll Learn

- Auto-generate documentation from device configs using an LLM (Large Language Model)
- Create network topology diagrams with AI
- Build a documentation pipeline that stays in sync with your network

**The Problem**: Documentation scattered across SharePoint (outdated), wikis (conflicting), tribal knowledge (gone when Bob leaves).

**The Solution**: Generate docs directly from configs -- always accurate because they come from the source of truth.

### How This Works (If You're New to AI)

We send device configuration text to Claude (Anthropic's AI) via an API call. Claude reads the config -- like a senior engineer would -- and returns structured documentation (JSON, Markdown tables). The key insight: **LLMs are very good at reading unstructured text and extracting structured data from it.**

**Networking analogy**: Think of it like SNMP, but instead of polling OIDs for structured MIB data, you send a config file and describe what you want in plain English.

## Setup

First, install the Anthropic Python SDK and set your API key.
You can get a free API key at [console.anthropic.com](https://console.anthropic.com/).

In [ ]:
# Install the Anthropic Python SDK
# This is the official client library for calling Claude's API
!pip install -q anthropic

In [ ]:
# Set your API key securely
# The getpass function hides your key as you type (like a password prompt)
import os
from getpass import getpass

if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass('Enter your Anthropic API key: ')

print("API key configured. Ready to go.")

## Demo 1: Generate Device Documentation from Config

We'll take a sample Cisco IOS router config and have Claude extract a structured overview,
interface table, and routing documentation from it.

**What's happening under the hood**: We send the config as text in an API request. The prompt
tells Claude exactly what to extract and what format to return it in (JSON). This is called
"prompt engineering" -- writing clear instructions so the AI returns useful, structured output.

In [ ]:
from anthropic import Anthropic
import json
import re
from datetime import datetime

# Initialize the Anthropic client.
# It automatically uses the ANTHROPIC_API_KEY environment variable we set above.
client = Anthropic()

# Which Claude model to use. Sonnet offers the best balance of quality and cost.
# For production with many devices, you could use "claude-haiku-4-20250514" for
# simple extraction tasks (10x cheaper).
MODEL = "claude-sonnet-4-20250514"

# Helper function: safely extract JSON from Claude's response.
# LLMs sometimes wrap JSON in markdown code fences or add explanation text.
def parse_json_response(text):
    """Extract JSON from an LLM response, handling common formatting quirks."""
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    # Try markdown code fence
    match = re.search(r'```(?:json)?\s*([\s\S]*?)```', text)
    if match:
        try:
            return json.loads(match.group(1))
        except json.JSONDecodeError:
            pass
    # Try finding JSON boundaries
    start, end = text.find('{'), text.rfind('}')
    if start != -1 and end != -1:
        try:
            return json.loads(text[start:end + 1])
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse JSON from response: {text[:200]}...")

# ---------------------------------------------------------------------------
# Sample Cisco IOS router configuration
# In production, you'd read this from a file or pull it from a device via
# Netmiko/NAPALM. Here we use a realistic example inline.
# ---------------------------------------------------------------------------
sample_config = """
hostname router-core-01
!
interface Loopback0
 ip address 192.168.1.1 255.255.255.255
!
interface GigabitEthernet0/0
 description Uplink to ISP-A (Lumen)
 ip address 203.0.113.1 255.255.255.252
 no shutdown
!
interface GigabitEthernet0/1
 description Trunk to DC-SPINE-01
 ip address 10.0.1.1 255.255.255.0
 no shutdown
!
interface GigabitEthernet0/2
 description Management OOB
 vrf forwarding MGMT
 ip address 172.16.0.1 255.255.255.0
 no shutdown
!
router ospf 1
 router-id 192.168.1.1
 network 10.0.0.0 0.0.255.255 area 0
 network 192.168.1.1 0.0.0.0 area 0
 passive-interface default
 no passive-interface GigabitEthernet0/1
!
router bgp 65001
 bgp router-id 192.168.1.1
 bgp log-neighbor-changes
 neighbor 203.0.113.2 remote-as 65002
 neighbor 203.0.113.2 description ISP-A_Lumen
 neighbor 203.0.113.2 route-map ISP-A-IN in
 neighbor 203.0.113.2 route-map ISP-A-OUT out
!
ip prefix-list ADVERTISE-TO-ISP seq 10 permit 198.51.100.0/24
!
route-map ISP-A-OUT permit 10
 match ip address prefix-list ADVERTISE-TO-ISP
!
route-map ISP-A-IN permit 10
 set local-preference 200
!
ip access-list extended MANAGEMENT_ACCESS
 permit tcp 10.0.0.0 0.0.255.255 any eq 22
 permit tcp 172.16.0.0 0.0.0.255 any eq 22
 deny ip any any log
!
line vty 0 4
 access-class MANAGEMENT_ACCESS in
 transport input ssh
"""

print("Sample config loaded (router-core-01)")
print(f"Config length: {len(sample_config)} characters")
print(f"Config preview:\n{sample_config[:200]}...")

In [ ]:
def generate_device_overview(config: str, hostname: str) -> dict:
    """Generate high-level device documentation from a config.

    This function sends the config to Claude with a prompt that asks
    it to extract key facts (role, IPs, protocols) and return them
    as structured JSON.

    The key parameters in the API call:
    - model: which Claude model to use
    - max_tokens: maximum response length (1500 tokens ~ 1000 words)
    - temperature: 0 = deterministic (same input -> same output)
    - messages: the conversation (just one user message here)
    """

    prompt = f"""Analyze this network device configuration and create documentation.

Device: {hostname}
Configuration:
{config}

Extract and document:
1. Device role (core router, access switch, firewall, etc.)
2. Management IP address (Loopback0 or management interface)
3. Routing protocols in use (BGP, OSPF, EIGRP, static)
4. Key features enabled (HSRP, VRF, QoS, route-maps, etc.)
5. Interface count and types
6. Notable configurations or policies

Return ONLY valid JSON with no additional text:
{{
    "hostname": "device name",
    "role": "device role",
    "management_ip": "IP address",
    "routing_protocols": ["list of protocols"],
    "key_features": ["list of features"],
    "interface_summary": "summary of interfaces",
    "notable_config": "anything important to know"
}}"""

    response = client.messages.create(
        model=MODEL,
        max_tokens=1500,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )

    # Parse the response -- using our safe parser in case Claude
    # wraps the JSON in code fences or adds text around it
    doc_data = parse_json_response(response.content[0].text)
    doc_data['generated_at'] = datetime.now().isoformat()

    return doc_data

# Generate overview
print("Sending config to Claude for analysis...")
print("(This makes one API call -- typically 1-3 seconds)\n")
overview = generate_device_overview(sample_config, "router-core-01")

print("=" * 60)
print("DEVICE OVERVIEW (returned as structured JSON)")
print("=" * 60)
print(json.dumps(overview, indent=2))

In [ ]:
def generate_interface_table(config: str) -> str:
    """Generate a markdown table of all interfaces.

    Similar to 'show ip interface brief' but enriched with
    descriptions and VLAN/VRF assignments from the config.
    """

    prompt = f"""Extract all interfaces from this network device config and create a markdown table.

Config:
{config}

Create a table with these exact columns:
| Interface | IP Address | Status | Description | VLAN/VRF |

Rules:
- Include ALL interfaces: physical, loopback, tunnel, VLAN, port-channel
- If no IP address is configured, show "L2" or "N/A"
- For status, use "up" if "no shutdown" or "down" if "shutdown"
- If no description is configured, show "-"

Return ONLY the markdown table, no other text."""

    response = client.messages.create(
        model=MODEL,
        max_tokens=2000,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

# Generate interface table
print("Generating interface table...\n")
interface_table = generate_interface_table(sample_config)

print("=" * 60)
print("INTERFACE TABLE")
print("=" * 60)
print(interface_table)

In [ ]:
def generate_routing_documentation(config: str) -> str:
    """Document routing configuration from a device config.

    Produces structured documentation of all routing protocols,
    static routes, redistribution, and route policies.
    """

    prompt = f"""Document the routing configuration from this network device.

Config:
{config}

Create documentation covering these sections. If a section does not
apply (e.g., no redistribution configured), write "None configured."

## Routing Protocols
- Which protocols are enabled (OSPF, BGP, EIGRP, IS-IS)
- Process IDs, AS numbers, area assignments
- Router IDs
- Neighbor statements and peer groups

## Static Routes
- Destination networks and next hops
- Administrative distance overrides
- Purpose (if evident from context)

## Route Redistribution
- What is redistributed into what
- Route-maps or filters applied

## Routing Policies
- Route-maps and their match/set clauses
- Prefix-lists and community-lists
- Access-lists that affect routing decisions

Format as clean markdown with sections and bullet points."""

    response = client.messages.create(
        model=MODEL,
        max_tokens=2500,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

# Generate routing docs
print("Generating routing documentation...\n")
routing_docs = generate_routing_documentation(sample_config)

print("=" * 60)
print("ROUTING DOCUMENTATION")
print("=" * 60)
print(routing_docs)

## Demo 2: Create Network Topology Diagrams from CDP/LLDP

Now we'll take raw CDP neighbor output and have Claude generate a
[Mermaid](https://mermaid.js.org/) diagram -- a text-based format that
renders as a visual topology in GitHub, Notion, Confluence, and many other tools.

**What this replaces**: Manually SSH'ing into each device, running `show cdp neighbors detail`,
and drawing boxes in Visio. This does the same thing automatically.

In [ ]:
# Sample CDP neighbor output -- this is what "show cdp neighbors detail" looks like.
# In production, you'd collect this from devices via Netmiko/NAPALM/Ansible.
cdp_output = """
Device ID: switch-dist-01.example.com
  IP address: 10.0.1.2
  Platform: cisco WS-C3850-24T, Capabilities: Switch IGMP
  Interface: GigabitEthernet0/1, Port ID (outgoing port): GigabitEthernet1/0/1
  Holdtime: 160 sec

Device ID: router-core-02.example.com
  IP address: 10.0.2.1
  Platform: Cisco ISR4451-X/K9, Capabilities: Router Switch IGMP
  Interface: GigabitEthernet0/2, Port ID (outgoing port): GigabitEthernet0/2
  Holdtime: 145 sec

Device ID: firewall-01.example.com
  IP address: 10.0.3.1
  Platform: Palo Alto Networks PA-5250, Capabilities: Router
  Interface: GigabitEthernet0/3, Port ID (outgoing port): ethernet1/1
  Holdtime: 120 sec
"""

print("Sample CDP output loaded (from router-core-01)")
print(f"Shows {cdp_output.count('Device ID:')} neighbors")

In [ ]:
def extract_neighbors_and_generate_diagram(cdp_output: str, local_device: str) -> str:
    """Extract neighbors from CDP output and generate a Mermaid topology diagram.

    We could do this in two steps (extract JSON, then generate diagram),
    but for simplicity we ask Claude to do both in one call.
    """

    prompt = f"""Analyze this CDP/LLDP output and create a network topology diagram.

Local Device: {local_device}
CDP Output:
{cdp_output}

Generate a Mermaid flowchart diagram showing:
1. All devices as nodes (use the short hostname, not FQDN)
2. All connections between them with interface labels
3. Use appropriate shapes:
   - Routers: rectangle [name]
   - Switches: stadium shape ([name])
   - Firewalls: hexagon {{{{name}}}}
4. Start with "graph TB" for top-to-bottom layout

Return ONLY the Mermaid syntax starting with "graph TB".
Do NOT wrap it in code fences."""

    response = client.messages.create(
        model=MODEL,
        max_tokens=1500,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )

    # Clean up any code fences the model might add
    import re
    text = response.content[0].text.strip()
    text = re.sub(r'^```(?:mermaid)?\s*', '', text)
    text = re.sub(r'\s*```$', '', text)
    return text

# Generate diagram
print("Generating topology diagram from CDP data...\n")
diagram = extract_neighbors_and_generate_diagram(cdp_output, "router-core-01")

print("=" * 60)
print("TOPOLOGY DIAGRAM (Mermaid syntax)")
print("=" * 60)
print(diagram)
print()
print("To view this diagram:")
print("  1. Paste into GitHub README or any Mermaid-compatible viewer")
print("  2. Or visit https://mermaid.live and paste it there")

## Demo 3: Complete Documentation Generator

Now let's combine all the pieces: generate a full device documentation page
that includes the overview, interface table, and routing details -- all
assembled into a single Markdown document.

In production, you'd run this for every device in your network and store
the results in Git (version-controlled documentation).

In [ ]:
def generate_complete_documentation(config: str, hostname: str) -> str:
    """Generate complete device documentation.

    Makes 3 separate API calls (overview, interfaces, routing) and
    assembles the results into a single Markdown document.

    Why 3 calls instead of 1? Each section gets focused attention
    from the model. If one call fails, the others still succeed.
    And it's easier to update individual prompts later.
    """

    print(f"Generating complete documentation for {hostname}...")

    # Get all sections
    print("  [1/3] Device overview...")
    overview = generate_device_overview(config, hostname)

    print("  [2/3] Interface table...")
    interfaces = generate_interface_table(config)

    print("  [3/3] Routing documentation...")
    routing = generate_routing_documentation(config)

    # Build complete doc
    doc = f"""# {hostname} - Device Documentation

**Generated**: {overview['generated_at']}
**Device Role**: {overview.get('role', 'Unknown')}
**Management IP**: {overview.get('management_ip', 'Unknown')}

---

## Overview

**Routing Protocols**: {', '.join(overview.get('routing_protocols', []))}
**Key Features**: {', '.join(overview.get('key_features', []))}

{overview.get('notable_config', '')}

---

## Interfaces

{interfaces}

---

## Routing Configuration

{routing}

---

*Auto-generated from device configuration using Claude AI.*
"""

    print("  Done!")
    return doc

# Generate full documentation (this makes 3 API calls)
full_doc = generate_complete_documentation(sample_config, "router-core-01")

print("\n" + "=" * 60)
print("COMPLETE DOCUMENTATION")
print("=" * 60)
print(full_doc)

## Try It With Your Own Config!

Replace the config below with a real device config from your network.
Then run the cell to generate documentation.

In [ ]:
# ---------------------------------------------------------------------------
# PASTE YOUR OWN DEVICE CONFIG BELOW
# ---------------------------------------------------------------------------
# Replace this placeholder with a real config from your network.
# Works with: Cisco IOS, IOS-XE, NX-OS, Arista EOS, Juniper JunOS, etc.
#
# SECURITY NOTE: Remove passwords, SNMP communities, and TACACS keys
# before pasting! Or use "show running-config | exclude password|secret|community"
# ---------------------------------------------------------------------------

your_config = """
hostname my-switch
!
interface Vlan10
 description Server VLAN
 ip address 10.10.10.1 255.255.255.0
 no shutdown
!
interface Vlan20
 description User VLAN
 ip address 10.10.20.1 255.255.255.0
 no shutdown
"""

your_hostname = "my-switch"  # Change this to your device's hostname

# Only generate if the config has real content (not just the placeholder)
if len(your_config.strip()) > 100:
    your_doc = generate_complete_documentation(your_config, your_hostname)
    print("\n" + your_doc)
else:
    print("Paste your device config in the 'your_config' variable above,")
    print("then run this cell again to generate documentation.")

## Key Takeaways

### What We Built
1. **Device Overview** -- Role, management IP, protocols, and features extracted as structured JSON
2. **Interface Table** -- All interfaces in a clean Markdown table (like an enhanced `show ip int brief`)
3. **Routing Documentation** -- Protocols, neighbors, policies documented in readable Markdown
4. **Topology Diagram** -- Mermaid diagram generated from CDP/LLDP data (no Visio needed)

### How It Works (The AI Part)
- We send **device config text** to Claude via an **API call**
- The **prompt** tells Claude what to extract and what format to use
- Claude returns **structured output** (JSON or Markdown) that we assemble into docs
- Setting `temperature=0` makes output **deterministic** (same config = same docs every time)

### Production Tips
- **Automate it**: Run on a schedule (daily/weekly) or trigger on config changes in Git
- **Version control**: Store generated docs in Git to track what changed and when
- **CI/CD integration**: Use GitHub Actions to regenerate docs when configs are committed
- **Cost control**: Use Haiku for simple extraction (~$0.001/device), Sonnet for complex analysis (~$0.01/device)
- **Sanitize configs**: Strip passwords and secrets before sending to the API

### Watch Out For
- **Stale configs** -- Make sure you're documenting the *current* running config, not a backup from last month
- **API costs at scale** -- Only regenerate docs for devices whose configs actually changed
- **Sensitive data** -- Always sanitize configs before sending to any external API

## Next Steps
- **Chapter 14**: RAG Fundamentals -- Make this documentation *searchable* with AI
- Try generating docs for your own devices using the "Try It" cell above
- Explore the standalone Python scripts in the `Code/` directory for production use